# Gitlab

In [1]:
from nornir.plugins.tasks.version_control import gitlab
print(gitlab.__doc__)


    Exposes some of the Gitlab API functionality for operations on files
    in a Gitlab repository.

    Example:

        nornir.run(files.gitlab,
                   action="create",
                   url="https://gitlab.localhost.com",
                   token="ABCD1234",
                   repository="test",
                   filename="config",
                   ref="master")

    Arguments:
        dry_run: Whether to apply changes or not
        url: Gitlab instance URL
        token: Personal access token
        repository: source/destination repository
        filename: source/destination file name
        content: content to write
        action: ``create``, ``update``, ``get``
        branch: destination branch
        destination: local destination filename (only used in get action)
        ref: branch, commit hash or tag (only used in get action)
        commit_message: commit message

    Returns:
        Result object with the following attributes set:
            * 

## Example 1 : create a file in a git repository on a gitlab server

In this example we will create a new file in a git repository on a gitlab server.

The contents that we will write to the file is a arbitrary string, in a real world scenario this could be the running configuration of a device that we fetched using napalm or through another method.

First let's import the necessary methods & tasks, then we will create a variable called `content` which is an arbitrary string.

In [1]:
from nornir import InitNornir
from nornir.plugins.tasks.version_control import gitlab
from nornir.plugins.tasks.commands import remote_command
from nornir.plugins.functions.text import print_result

inventory = {
    "plugin": "nornir.plugins.inventory.simple.SimpleInventory",
    "options": {
            "host_file": "gitlab_data/inventory/hosts.yaml"
    }
}

n = InitNornir(inventory=inventory)

content = """127.0.0.1\t\tlocalhost
255.255.255.255\tbroadcasthost
::1\t\tlocalhost
"""

And create a new file called `hosts` in the repository `test` on the `master` branch.

In [2]:
import requests_mock
from functools import wraps

def wrap_gitlab(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        with requests_mock.Mocker() as m:
            if kwargs.get("ref", None):
                kwargs["branch"] = kwargs["ref"]
            m.post(url=f"{kwargs['url']}/api/v4/projects/{kwargs['repository']}/repository/files/{kwargs['filename']}", status_code=201)
            m.get(url=f"{kwargs['url']}/api/v4/projects/{kwargs['repository']}/repository/files/{kwargs['filename']}?ref={kwargs['branch']}",status_code=200, json={"content":"MTI3LjAuMC4xCQlsb2NhbGhvc3QKMjU1LjI1NS4yNTUuMjU1CWJyb2FkY2FzdGhvc3QKOjoxCQls\nb2NhbGhvc3QK\n"})
            m.put(url=f"{kwargs['url']}/api/v4/projects/{kwargs['repository']}/repository/files/{kwargs['filename']}", status_code=200)
            return f(*args, **kwargs)
    return wrapper

gitlab = wrap_gitlab(gitlab)

In [3]:
result = n.run(
    gitlab,
    action="create",
    url="http://localhost:8080",
    token = "SuperSecretToken",
    repository="test",
    branch="master",
    filename="hosts",
    #content=results["dev5.no_group"][0]
    content=content,
    commit_message="Nornir is AWESOME!"
)

The result of the task shows us a diff of the created `hosts` file and the content we provided.

In [4]:
print_result(result)

gitlab**************************************************************************
* alpine ** changed : True *****************************************************
vvvv gitlab ** changed : True vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
--- 

+++ hosts

@@ -0,0 +1,3 @@

+127.0.0.1		localhost
+255.255.255.255	broadcasthost
+::1		localhost
^^^^ END gitlab ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Example 2 : update an existing file in a git repository on a gitlab server

In this example we will update the contents of the hosts file that we created in the previous step. The new contents could come again from a remote host or device, but in this case we will use an arbitrary value for the new contents of the file.

In [5]:
result = n.run(
    gitlab,
    action="update",
    url="http://localhost:8080",
    token="SuperSecretToken",
    repository="test",
    branch="master",
    filename="hosts",
    content=f"{content}8.8.8.8\t\tgoogledns",
    commit_message="Added new line to hosts file"
)

The result of the task should show us a diff of the changes that we made.

In [6]:
print_result(result)

gitlab**************************************************************************
* alpine ** changed : True *****************************************************
vvvv gitlab ** changed : True vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
--- hosts

+++ hosts

@@ -1,3 +1,4 @@

 127.0.0.1		localhost
 255.255.255.255	broadcasthost
 ::1		localhost
+8.8.8.8		googledns
^^^^ END gitlab ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Example 3: get a file from a gitlab repository


In this example we will get/download a file from a repository in gitlab. The contents of this file could be a staged configuration of a device or a service on a device. This configuration could then be pushed to the device.

In our example we will download the file `hosts` from the `master` branch and save it as `/tmp/hosts`.
The `ref` parameter can also be a commit hash or tag.

In [7]:
!rm -f /tmp/hosts
result = n.run(
    gitlab,
    action="get",
    url="http://localhost:8080",
    token="SuperSecretToken",
    repository="test",
    ref="master",
    filename="hosts",
    destination="/tmp/hosts"
)


The result should show us a new file `/tmp/hosts` being created on the local system.

In [8]:
print_result(result)

gitlab**************************************************************************
* alpine ** changed : True *****************************************************
vvvv gitlab ** changed : True vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
--- /tmp/hosts

+++ /tmp/hosts

@@ -0,0 +1,3 @@

+127.0.0.1		localhost
+255.255.255.255	broadcasthost
+::1		localhost
^^^^ END gitlab ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
